# Amazon VPC Security & Monitoring
Two of the flagship benefits of a VPC are the security it can provide for a group of cloud resources, and the comprehensive control of the network surrounding those resources. This notebook will explore these features in depth, discussing the different layers of security available for a VPC, and the various network monitoring options too.

## VPC Security

### Security Groups
>*Security Groups* are collections of rules which control the network traffic in and out of the **instances** they are associated with, such as EC2 or RDS instances. 

These are the important characteristics of Security Groups you need to know:

- When you create a VPC, a Security Group is created by default, which is set to allow all traffic in and out of the instances it is associated with

- They can only be made to allow traffic in or out of an instance, not deny

- They are *stateful*, which means if a request is allowed out by a particular rule, then the corresponding response is automatically allowed access as well

### NACLs (Network Access Control Lists)
>*NACLs* are collections of rules which control the network traffic in and out of the **subnets** they are associated with.

These are the important characteristics of NACLs you need to know:

- Each subnet must be associated with one NACL. New subnets get a default NACL, which accepts all inbound and outbound traffic.

- NACLs can be customised to allow or deny traffic

- NACL rules are numbered. Rules are evaluated in order of their rule number, so lower numbers get a higher precedence. For example, traffic will be allowed in to an instance if there is a rule of number 100 which allows it, even if there is another rule of number 200 which denies it. 

- They are *stateless*, which means that every request and response in and out of a subnet is checked against the rules exclusively

#### NACLs and Ephemeral Ports
For an instance and a target to establish a connection they must use ports: the instance connects to a *defined port* on the target, and expects a response on an *ephemeral port*. This is important to remember when creating NACL rules, since NACLs are stateless. 

When creating rules for inbound traffic we need to open the required ephemeral port range so that a response can be received by the instance. The port range varies depending on the instance's operating system: Linux machines use 32768-61000, most recent Windows versions use 49152-65535, for example.

### Security Group and NACL Rules
Security Group and NACL rules are created as either inbound or outbound rules, specifying for each the following attributes:

- **Type:** This is the protocol you want to allow in or out of your instance. For example, some common protocols to choose are HTTP/HTTPS for web traffic, SSH for secure communication, or TCP. 

- **Protocol:** This will be greyed out and filled in for you, unless you select *Custom Protocol* in the Type dropdown list, in which case you can manually type the protocol yourself
- **Port range:** A specific port number or range of port numbers traffic is allowed to access. Again this might be auto-populated depending on what protocol type you chose. 
- **Source/Destination:** For inbound rules, the source of the traffic. For outbound rules, the destination where you want the traffic to go. This can be either your own IP address, any IP address, or a custom IP address which you then enter in the next box.
- **Description:** An optional description of what the rule is and why it is there
- **Rule Number (for NACLs only):** The order of precedence for which rules to evaluate first. AWS recommends you number rules using increments of 100, so that you can add more in between if necessary. 

The image below is an example of adding outbound rules to a Security Group. In this particular case the rule is set to allow HTTP traffic out to any IPv4 address. HTTP automatically uses a TCP connection and comes from port 80 of the instance.

<p align="center"> <img src="images/SGs.png" height="267" width="1733"/> </p>

### Hands-On: Creating and Configuring Security Groups and NACLs [WIP]
In this Hands-On we are going to set up the security so that two EC2 instances in separate subnets can communicate with eachother. This will involve creating both instances, then creating and configuring Security Group and NACL inbound and outbound rules for each. We will be using only free tier resources. 

#### Step 1: Create and Configure The EC2 instances
- Log in to your AWS management console and navigate to EC2 using the search bar. Make sure the region is set to Ireland.

- Press *Launch instance*
- Enter a name for your instance. Make sure the Instance type is *t2.micro* and says *Free tier eligible*.
- Create new key pair, making sure it is a `.pem` file. It will automatically download when you press *Create key pair*, make sure it is somewhere safe on your local computer.
- In *Network settings* select a VPC and public subnet (you can use one from a previous Hands-on)
- Select *Enable* for *Auto-assign public IP*. In order to connect to our EC2s from the AWS console they need to have public IP addresses assigned to them.
- Make sure that *Create security group* is selected, but *Allow SSH traffic from* is deselected (we are going to manually configure this in a later step)
- Leave everything else the same then press *Launch instance*
- Complete this whole step once more to create another EC2 instance, making sure to call it something different and placing it in a separate subnet

#### Step 2: Ensure public subnets are routed correctly
- If you are using public subnets from a previous Hands-on then double check you still have an Internet Gateway routed to the subnet

- If you have created new subnets for this exercise, then you will have to also create an Internet Gateway to make the subnets public. Revisit (this)[] notebook to find out how.

#### Step 3: Create Security Group for instance-1
- Navigate to the VPCs console, then Security Groups on sidebar
**Inbound Rules**

**Outbound Rules**

#### Step 4: Create NACL for instance-1
**Inbound Rules**

**Outbound Rules**

#### Step 5: Create Security Group for instance-2
**Inbound Rules**

**Outbound Rules**

#### Step 5: Create NACL for instance-2
**Inbound Rules**

**Outbound Rules**

### Step 6: Test the set up


### Firewalls  
A *firewall* is a network security device which monitors incoming and outgoing traffic and decides whether to allow or block specific traffic based on a defined set of security rules. This section outlines two of the main firewall services available for AWS.

#### AWS Network Firewall
>*AWS Network Firewall* is an intrusion detection and prevention service for Amazon VPCs.

It is made up of the following components:

- **Rule Group:** A reusable collection of criteria which decide whether packets in a traffic flow are allowed to pass or drop. For example, if the traffic came from a particular range of IP addresses. Rule groups are either stateless or stateful: stateless rule groups evaluate packets in isolation, while stateful rule groups evaluate them in the context of their traffic flow. 

- **Firewall Policy:** A reusable collection of Rule Groups, along with some policy-level behaviour such as what happens if traffic breaks midstream, and default actions for both stateless and stateful traffic.
- **Firewall:** Connects the Firewall Policy to the VPC. Each Firewall requires at least one Firewall Policy. The Firewall additionally defines settings like how to monitor and log network traffic information.

#### AWS Web Application Firewall (WAF)
>*AWS WAF* is a network protection and monitoring service specifically designed for HTTP/HTTPS traffic to and from web application resources. 

It works by defining traffic rules and then associating them with web application resources. A rule describes traffic patterns to look for in requests and to specify the actions to take on matching requests. These actions can be:

- Allow the requests to go to the protected resource for processing and response
- Block the requests
- Count the requests
- Run CAPTCHA or challenge checks against requests to verify human users and standard browser use

AWS WAF is made up of the following components:

- **Rules:** Rules contain statements that define the traffic inspection criteria, then the action to take if a request meets the criteria. For example:
    - Defining IP addresses or ranges which requests come from
    - The country or location which requests originate from
    - You can check for specific known malicious request attributes such as the query string, the request header value or body values

- **Rule Groups:** Reusable collections of rules. You can create your own rule groups or use predefined ones from the AWS Marketplace.
- **Web ACLs (Access Control Lists):** Collections of rule groups, where you decide in which order the rules and rule groups should be evaluated

## VPC Monitoring
>At a high level, *network monitoring* or *logging* is the process of documenting every event that takes place on a device. From a networking perspective, this allows you to ensure security by observing the flow of traffic in and out of your network at a fine granularity, in addition to optimising your resources. It is a crucial part of developing a robust network infrastructure.

When discussing network monitoring or logging it is common to use some of the following metrics:

- **Throughput:** The rate of traffic, in bytes per second, passing through a device during a specific time period

- **Uptime**: Also known as *availability*, this is the amount of time that a network device successfully sends and receives data
- **Bandwidth:** The amount of data, in bytes, that is currently being sent or received by a device. Engineers track both the volume of traffic being sent, and the percentage of total bandwidth that is being utilised.
- **CPU utilisation:** Also sometimes called *compute*. This metric is how much a device has used its computational capacity to process input, store data, and create output.
- **Interface errors/discards:** These are errors on the receiving device that cause a network interface to drop a data packet. Interface errors and discards can stem from configuration errors or bandwidth issues, for example.
- **IP metrics:** IP metrics such as *latency* can measure the speed and efficiency of connections between devices.

The remainder of this section deals with some of the more common VPC monitoring services for AWS.

### AWS CloudWatch
>*AWS CloudWatch* is a service which monitors any of your AWS resources or applications hosted on AWS resources, in real time. It is a metrics repository which you can use to collect, organise and visualise metrics in graph and dashboard form. You can also create alarms to watch metrics and send automatic notifications or automatically change resources when a threshold is breached. For example, you could monitor CPU usage of an EC2 instance then use that data to decide whether to launch a new instance to handle increased load.

These are the core concepts fundamental to understanding AWS CloudWatch:

- **Namespaces:** A namespace is a container for metrics. There is a different, isolated namespace for each AWS service that produces CloudWatch-supported metrics, refer to [this](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/aws-services-cloudwatch-metrics.html) documentation for the full list. Namespaces are denoted as "AWS/<service>", so metrics for an EC2 instance would be from the AWS/EC2 namespace
- **Metrics:** The most fundamental concept of CloudWatch, a metric is a stream of data points produced by a single variable over time. The SSD storage being used for an RDS instance usage, for example.
- **Dimensions**: Key/value pairs that help to categorise metrics. You can assign multiple dimensions to each metric, which then allows you to split the same metric into subcategories, which is useful for aggregating statistics with them later. 
    
    For example, you have a prod and a dev EC2 instance in two different regions, but you want to calculate their CPU usage separately. To achieve this you would assign them each a dimension for their server type, and for their region, which would ultimately look like this:

    ```
    Dimensions: Server=Prod, Region=eu-west-1, Unit: CPU Usage, Timestamp: 2016-10-31T12:30:00Z, Value: 10%
    Dimensions: Server=Beta, Region=eu-west-1, Unit: CPU Usage, Timestamp: 2016-10-31T12:31:00Z, Value: 11%
    Dimensions: Server=Prod, Region=uk-south,  Unit: CPU Usage, Timestamp: 2016-10-31T12:32:00Z, Value: 95%
    Dimensions: Server=Beta, Region=uk-south,  Unit: CPU Usage, Timestamp: 2016-10-31T12:33:00Z, Value: 97%
    ```

- **Resolution:** The frequency at which the metric data points are gathered. This is either *Standard* or *High* resolution. Standard resolution is data gathered at a one minute time interval, high resolution is at one second.
- **Statistics:** Aggregations of metrics over specific periods of time
- **Alarms:** An alarm watches a single metric and can automatically initiate actions depending on the value of the metric


### AWS CloudTrail
>*AWS CloudTrail* is an event tracking system for an AWS account. Actions taken by a user, role, or an AWS service are recorded as events in CloudTrail. Just like CloudWatch you can organise and visualise the events in a dashboard.

CloudTrail provides three ways to record events:

- **Event history:** A searchable record of the past 90 days of events in an AWS region. This is a free feature, and you automatically have access to CloudTrail Event History by default when you create an AWS account.

- **CloudTrail Lake:** A managed data lake for capturing, storing and analysing user activity. You can keep the data for up to 10 years or up to 7 years, depending on which pricing plan you choose. You can also import existing CloudTrail logs from S3 buckets into CloudTrail Lake to use the dashboarding features.

- **Trails:** Captures and records events, storing them in an S3 bucket, with optional delivery to CloudWatch logs or other external security and monitoring solutions


### VPC Flow Logs
>*VPC Flow Logs* are a component of the VPC ecosystem that allow you to capture very detailed information about traffic going to and from a VPC, subnets, and at instance level. 

The logs can be sent to S3, CloudWatch logs or other data streaming and monitoring services. Flow logs collect data from outside the path of the traffic, so they do not affect network throughput or latency.

Use cases of flow logs:

- Troubleshooting connectivity issues
- Configuring overly restrictive or permissive Security Groups or NACLs
- Detecting malicious intrusions 

#### Flow Log Records
The image below is an example of what flow log records look like. 

In the first record, HTTP traffic (desination port 80, TCP protocol) containing 3 packets of data of size 140 bytes, from IP address `83.234.179.125` has been rejected access to private IP address `172.31.22.145`.

<p align="center"> <img src="images/flow-log-records.png" height="384" width="934"/> </p>

## Key Takeaways
- Security Groups allow you to control network traffic to and from specific instances. They are stateful, and can only be made to allow traffic, not deny.

- NACLs allow you to control network traffic at the subnet level. They are stateless and can allow or deny traffic.
- SG and NACL rules are defined as either inbound or outbound, specifying the protocol type, port range, source/destination, and a description. Additionally, NACL rules are numbered, and are evaluated in order of their numbers.
- AWS Network Firewall is an intrusion detection and monitoring service for VPCs. It consists of traffic rules, grouped together into Rule Groups, which are then assigned to a Firewall Policy.
- AWS WAF is a network detection and monitoring service for HTTP/HTTPS traffic to and from web application resources. A WAF consists of a web ACL, which is a collection of rule groups. 
- Monitoring is a very important part of a complete and robust VPC infrastructure. Some metrics which can be discussed are: throughput, uptime, bandwidth, CPU utilisation, interface errors and IP metrics.
- AWS Cloudwatch is a monitoring and analysis service for AWS resources, which you can use to notify you of instance state changes and automatically make changes to the instance itself. It's core components are: namespaces, metrics, dimensions, resolution, statstics and alarms. 
- Flow logs are a fundamental component of Amazon VPCs which collect thoroughly detailed traffic data, and can deliver it to log storage and analysis services